In [1]:
import pandas as pd
import json

In [2]:
from ipywidgets import IntProgress
from IPython.display import display

### load data

In [3]:
def load_from_telegram(telegram_file_name, user = ''):
    with open(telegram_file_name) as f:
        file_data = json.load(f)

    df = pd.DataFrame(columns=['text'])
    for msg in file_data['messages']:    
        if type(msg['text'])==type('') and len(msg['text'])>20 and len(msg['text'])<200:
            if user == '' or msg['from']==user:
                df = df.append({'text':msg['text']}, ignore_index=True)
    df = df[~df.duplicated(subset=['text'], keep=False)]
    df.to_csv('data.csv')
    return df

In [4]:
def restore_data_from_file():
    return pd.read_csv('data.csv')

In [5]:
#df = load_from_telegram('telegram.json','Vladimir Korotkov')
df = restore_data_from_file()
len(df)

5987

### prepare model

In [6]:
from deeppavlov import build_model, configs

In [7]:
model = build_model(configs.squad.squad_ru_rubert_infer, download=False)

[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/alex/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/alex/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
/home/alex/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/alex/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type

2020-10-26 21:57:31.642 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /home/alex/.deeppavlov/models/squad_ru_bert/model_rubert]
I1026 21:57:31.642025 140405655562048 tf_model.py:51] [loading model from /home/alex/.deeppavlov/models/squad_ru_bert/model_rubert]
W1026 21:57:31.643726 140405655562048 deprecation_wrapper.py:119] From /home/alex/.local/lib/python3.6/site-packages/deeppavlov/core/models/tf_model.py:54: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.



In [37]:
def get_answer(df,question):
    
    # select phrases
    pre_words = question.replace("?","").split(" ")
    words =[]
    for word in pre_words:
        if len(word)>2:
            words.append(word)    
    if len(words)==0:
        return 'Слишком короткий запрос'    
    
    df = df[df.text.str.contains('|'.join(words), regex = True)]
    
    if len(words)==0:
        return 'Нет данных'
    
    data = df.text.values.tolist()
    
    prgBar = IntProgress(min = 0, max = len(data))
    display(prgBar)
    
    answers = pd.DataFrame(columns=['answer','score'])
    for theme in data:
        res = model([theme], [question])
        answers = answers.append({'answer':res[0][0], 'score':res[2][0]}, ignore_index=True)            
        prgBar.value = prgBar.value + 1
            
    max_score = answers.score.max()    
    return answers[answers.score==max_score].iloc[0].answer

In [31]:
answer = get_answer(df,'Нужно ли сортировать мусор?')
answer

IntProgress(value=0, max=1697)

'придётся'